# Projeto de web scraping: análise de dados do site MAL
#### Esse projeto tem como objetivo coletar dados relevantes, disponíveis publicamente no site My Anime List, efetuar uma limpeza nos dados coletados e montar uma base de dados explorável. 

## Etapa 1 - Coleta dos dados no ranking de Top Animes:

### Preparação para a coleta:
#### Instalando as bibliotecas:

In [148]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import requests
import re
import numpy as np
import datetime as dt

#### Carregando a página:

In [33]:
url = 'https://myanimelist.net/topanime.php'
pg = bs(requests.get(url).content)

#### Selecionando os dados utéis:

In [196]:
dicionario_dados = {'Título':[],'Pontuação':[],'Rank':[]} #Dicionário modelo para armazenar os dados posteriormente 

### Coletando informações:

#### Títulos:

In [197]:
titles = pg.find('table').select('h3 a')
titulos = [title.text for title in titles]

#### Pontuação:

In [198]:
scores = pg.find('table').find_all(text=re.compile('^([0-9]\.[0-9]{2})$'))
pontuacoes = [float(score) for score in scores] # Transformando em float

#### Rank:

In [199]:
ranks = pg.find('table').select('.rank.ac')
ranques = [int(rank.get_text(strip=True)) for rank in ranks] # Transformando os valores em inteiros

### Armazenando os dados:

In [200]:
dicionario_dados['Título'] = titulos
dicionario_dados['Pontuação'] = pontuacoes
dicionario_dados['Rank'] = ranques

## Etapa 2 - Coleta de dados nas páginas por anime:

### Coletando os links para escalonamento da extração de dados:

#### Capturando os links das páginas:

In [201]:
urls = pg.find('table').select('h3 a')
links = [url['href'] for url in urls]

#### Capturando o link para passagem de página:

In [34]:
next_link = pg.find(class_='link-blue-box next')['href']
dominio = 'https://myanimelist.net/topanime.php'
next_link_completo = dominio + next_link

### Coletando dados em páginas individuais:

#### Carregando a página:

In [35]:
fma_pg = requests.get('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood')
fma_soup = bs(fma_pg.content)

#### Selecionando os dados úteis ao projeto:

In [204]:
#Esses foram os dados escolhidos das páginas individuais dos animes
dicionario_dados['Tema'] = [] #Dicionário modelo para armazenar dados posteriormente
dicionario_dados['Gêneros'] = []
dicionario_dados['Estúdio'] = []
dicionario_dados['Demografia'] = [] 
dicionario_dados['Origem'] = []
dicionario_dados['Episódios'] = []
dicionario_dados['Estreou'] = []
dicionario_dados['Horário exibido'] = []
dicionario_dados['Status'] = []
dicionario_dados['Classificação'] = [] 
dicionario_dados['Tipo'] = []

#### Coletando os dados:

##### Temas:

In [36]:
tema = fma_soup.find('span',text='Theme:')
temas = tema.parent.find_all(itemprop='genre')
tema = [tema.text for tema in temas]
tema

['Military']

##### Gêneros:

In [38]:
genero = fma_soup.find('span',text=re.compile('(Genres:|Genre:)'))   
generos = genero.parent.find_all(itemprop='genre')
genero = [genero.text for genero in generos]
genero

['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy']

##### Estúdios:

In [207]:
estudio = fma_soup.find(text='Studios:') #
estudio.parent.next_sibling.next_sibling.text

'Bones'

   ##### Demografia:

In [208]:
demografia = fma_soup.find(text='Demographic:') #
demografia.parent.next_sibling.next_sibling.text

'Shounen'

##### Origem:

In [209]:
origem = fma_soup.find(text='Source:') 
origem.parent.next_sibling.get_text(strip=True)

'Manga'

##### Episódios:

In [210]:
episodios = fma_soup.find(text='Episodes:')
episodios.parent.next_sibling.get_text(strip=True)

'64'

##### Estreias:

In [211]:
estreia = fma_soup.find(text='Premiered:') 
estreia.parent.next_sibling.next_sibling.text

'Spring 2009'

##### Horários exibidos:

In [212]:
horario = fma_soup.find(text='Broadcast:')
horario.parent.next_sibling.get_text(strip=True)


'Sundays at 17:00 (JST)'

##### Status:

In [213]:
status = fma_soup.find('span',text='Status:')
status.next_sibling.get_text(strip=True)

'Finished Airing'

##### Classificação:

In [214]:
r = fma_soup.find(text='Rating:')
r.next.get_text(strip=True)

'R - 17+ (violence & profanity)'

##### Tipos:

In [215]:
tipo = fma_soup.find(text='Type:')
tipo.parent.next_sibling.next_sibling.text

'TV'

### Definindo funções para a automatização do processo:

#### Funções referentes a página completa:

In [216]:
def extrair_pg(url): 
    try:
        pg_bruta = requests.get(url)
        pg = bs(pg_bruta.content)
        return pg
    except:
        print(f'Não foi possível acessar o link {url}')

In [217]:
def extrair_titulos(pg):
    try:
        titles = pg.find('table').select('h3 a')
        titulos = [title.text for title in titles]
        return titulos
    except:
        print('Não foi possível extrair os títulos da página')

In [218]:
def extrair_scores(pg):
    try:
        scores = pg.find('table').find_all(text=re.compile('^([0-9]\.[0-9]{2})$'))
        pontuacoes = [float(score) for score in scores] 
        return pontuacoes
    except:
        print('Não foi possível extrair as pontuações da páginas')

In [219]:
def extrair_ranks(pg):
    try:
        ranks = pg.find('table').select('.rank.ac')
        ranques = [int(rank.get_text(strip=True)) for rank in ranks]
        return ranques
    except:
        print('Não foi possível extrair os ranks')

In [220]:
def extrair_nextlink(pg):
    try:
        next_link = pg.find(class_='link-blue-box next')['href']
        dominio = 'https://myanimelist.net/topanime.php'
        next_link_completo = dominio + next_link
        return next_link_completo
    except:
        print('Não foi possível extrair o link da próxima página')

In [221]:
def extrair_links(pg):
    try:
        urls = pg.find('table').select('h3 a')
        links = [url['href'] for url in urls]
        return links
    except:
        print('Não foi possível extrair a lista de links')     

#### Função completa da lista:

In [222]:
def coletar_dados(url):
        pg = extrair_pg(url)
        titulos = extrair_titulos(pg)
        pontuacoes = extrair_scores(pg)
        ranques = extrair_ranks(pg)
        next_link = extrair_nextlink(pg)
        links = extrair_links(pg)
        
        dicionario_dados = {}
        dicionario_dados['Título'] = titulos 
        dicionario_dados['Pontuação'] = pontuacoes
        dicionario_dados['Rank'] = ranques
        
        dicionario_links = {}
        dicionario_links['Next'] = next_link
        dicionario_links['Lista'] = links
        return dicionario_dados, dicionario_links

#### Definindo funções para páginas individuais:

In [223]:
def extrair_tipo(pg):
    try:
        tipo = pg.find(text='Type:')
        tipo = tipo.parent.next_sibling.next_sibling.text
        return tipo
    except:
        return None

In [224]:
def extrair_tema(pg): 
    try:
        temas = pg.find('span',text=re.compile('(Theme:|Themes:)')) 
        temas = temas.parent.find_all(itemprop='genre')
        if len(temas) == 1:
            temas = temas[0].text
            return temas
        else:
            textos = ''
            for tema in temas:
                textos += ',' + ' ' + tema.text  
            textos = textos.replace(',','',1)
            return textos
    except:
        return None 

In [225]:
def extrair_genero(pg): 
    try:
        generos = pg.find('span',text=re.compile('(Genres:|Genre:)')) 
        generos = generos.parent.find_all(itemprop='genre')
        if len(generos) == 1:
            return generos[0].text
        else:
            textos = ''
            for genero in generos:
                textos += ',' + ' ' + genero.text  
            textos = textos.replace(',','',1)
            return textos      
    except:
        return None

In [226]:
def extrair_estudio(pg):
    try:
        estudio = pg.find(text='Studios:') 
        estudio = estudio.parent.next_sibling.next_sibling.text
        return estudio
    except:
        return None

In [227]:
def extrair_demografia(pg): 
    try:
        demografia = pg.find(text='Demographic:') 
        demografia = demografia.parent.next_sibling.next_sibling.text
        return demografia
    except:
        return None

In [228]:
def extrair_origem(pg):
    try:
        origem = pg.find(text='Source:') 
        origem = origem.parent.next_sibling.get_text(strip=True)
        return origem
    except:
        return None

In [229]:
def extrair_episodios(pg): 
    try:
        episodios = pg.find(text='Episodes:')
        episodios = episodios.parent.next_sibling.get_text(strip=True)
        return int(episodios)
    except:
        return None 

In [230]:
def extrair_estreias(pg): 
    try:
        estreia = pg.find(text='Premiered:') 
        estreia = estreia.parent.next_sibling.next_sibling.text
        return estreia
    except:
        return None

In [231]:
def extrair_horarios(pg):
    try:
        horario = pg.find(text='Broadcast:')
        horario = horario.parent.next_sibling.get_text(strip=True)
        return horario
    except:
        return None

In [232]:
def extrair_status(pg):
    try:
        status = pg.find('span',text='Status:')
        status = status.next_sibling.get_text(strip=True)
        return status
    except:
        return None

In [233]:
def extrair_r(pg):
    try:
        r = pg.find(text='Rating:')
        r = r.next.get_text(strip=True)
        return r
    except:
        return None

#### Função completa da página:

In [234]:
def extrair_tudo(url):
    pg = extrair_pg(url)
    tema = extrair_tema(pg)
    genero = extrair_genero(pg)
    estudio = extrair_estudio(pg)
    demografia = extrair_demografia(pg)
    origem = extrair_origem(pg)
    episodios = extrair_episodios(pg)
    estreia = extrair_estreias(pg)
    horarios = extrair_horarios(pg)
    status = extrair_status(pg)
    rating = extrair_r(pg)
    tipo = extrair_tipo(pg)
    
    dicionario_dados = {}
    dicionario_dados['Tema'] = tema
    dicionario_dados['Gêneros'] = genero
    dicionario_dados['Estúdio'] = estudio
    dicionario_dados['Demografia'] = demografia 
    dicionario_dados['Origem'] = origem
    dicionario_dados['Episódios'] = episodios
    dicionario_dados['Estréia'] = estreia
    dicionario_dados['Horário exibido'] = horarios
    dicionario_dados['Status'] = status
    dicionario_dados['Classificação'] = rating 
    dicionario_dados['Tipo'] = tipo
    return dicionario_dados

### Funções finais:

In [235]:
def formar_df(dict1,dict2):
    x = pd.DataFrame(dict1)
    y = pd.DataFrame(dict2)
    z = pd.concat([x,y],axis=1)
    return z

In [236]:
def coleta_completa(url):
    dicionario_dados, dicionario_links = coletar_dados(url)
    dict_list = []
    links = dicionario_links['Lista']
    next_link = dicionario_links['Next']
    for link in links:
        dicionario = extrair_tudo(link)
        dict_list.append(dicionario)
        time.sleep(1)
    df = formar_df(dict_list,dicionario_dados)
    return df,next_link

### Coletando todos os dados de páginas na 1° lista:

In [237]:
df_1,next_link = coleta_completa('https://myanimelist.net/topanime.php?limit=0')

In [250]:
df_1.head(3)

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação,Rank
0,Military,"Action, Adventure, Comedy, Drama, Fantasy",Bones,Shounen,Manga,64.0,Spring 2009,Sundays at 17:00 (JST),Finished Airing,R - 17+ (violence & profanity),TV,Fullmetal Alchemist: Brotherhood,9.15,1
1,"Military, Super Power","Action, Drama, Fantasy, Mystery",MAPPA,Shounen,Manga,12.0,Winter 2022,Mondays at 00:05 (JST),Currently Airing,R - 17+ (violence & profanity),TV,Shingeki no Kyojin: The Final Season Part 2,9.13,2
2,Psychological,"Drama, Sci-Fi, Suspense",White Fox,None,Visual novel,24.0,Spring 2011,Wednesdays at 02:05 (JST),Finished Airing,PG-13 - Teens 13 or older,TV,Steins;Gate,9.09,3
3,"Historical, Parody, Samurai","Action, Comedy, Sci-Fi",Bandai Namco Pictures,Shounen,Manga,51.0,Spring 2015,Wednesdays at 18:00 (JST),Finished Airing,PG-13 - Teens 13 or older,TV,Gintama°,9.09,4
4,"Military, Super Power","Action, Drama, Fantasy, Mystery",Wit Studio,Shounen,Manga,10.0,Spring 2019,Mondays at 00:10 (JST),Finished Airing,R - 17+ (violence & profanity),TV,Shingeki no Kyojin Season 3 Part 2,9.08,5


### Função para coleta de dados entre páginas com listas:

In [239]:
def formar_df_completo(url,n):
    links = [url]
    dfs = []
    for link in links:
        while len(links) <= n:
            df,link = coleta_completa(link)
            dfs.append(df)
            links.append(link)
            time.sleep(1)
    return dfs

### Coletando os dados dos primeiros 500 do ranking:

In [240]:
dfs = formar_df_completo('https://myanimelist.net/topanime.php?limit=0',10)

In [248]:
animes_df = pd.concat(dfs)
animes_df.head(3)

#### Salvando o dataframe:

In [259]:
animes_df.to_excel('df.xlsx',index=False)

## Etapa 2 -  Limpando o dataframe:

#### Analisando o dataframe:

In [11]:
df = pd.read_excel('df.xlsx',index_col='Rank') #Carregando os dados salvos
df.head(3)

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
1,Military,"Action, Adventure, Comedy, Drama, Fantasy",Bones,Shounen,Manga,64.0,Spring 2009,Sundays at 17:00 (JST),Finished Airing,R - 17+ (violence & profanity),TV,Fullmetal Alchemist: Brotherhood,9.15
2,"Military, Super Power","Action, Drama, Fantasy, Mystery",MAPPA,Shounen,Manga,12.0,Winter 2022,Mondays at 00:05 (JST),Currently Airing,R - 17+ (violence & profanity),TV,Shingeki no Kyojin: The Final Season Part 2,9.13
3,Psychological,"Drama, Sci-Fi, Suspense",White Fox,NaN,Visual novel,24.0,Spring 2011,Wednesdays at 02:05 (JST),Finished Airing,PG-13 - Teens 13 or older,TV,Steins;Gate,9.09


In [3]:
df.isna().sum() # Mostra a quantidade da valores nulos por coluna

Tema               151
Gêneros             11
Estúdio              4
Demografia         229
Origem               4
Episódios            7
Estréia            207
Horário exibido    207
Status               4
Classificação        4
Tipo                 9
Título               0
Pontuação            0
dtype: int64

###### Os dados de temporada de estréia e horário exibido totalizam 207 nulos por coluna, sendo 41% do total de linhas, porém esses dados se referem a filmes que não possuem horário exibido fixo e estreiam a parte da temporada de animes. Já demografia e tema tem valores nulos recorrentes, principalmente pela falta de dados nas páginas por anime.

#### Tratando os dados nulos:

In [12]:
nulos = df.loc[df['Estúdio'].isna() == True] # localizando os valores nulos recorrentes em mais de uma coluna
display(nulos)

df.drop([27,89,97,292],axis=0,inplace=True) # Retirando as linhas nulas por index

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gintama.: Shirogane no Tamashii-hen,8.82
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JoJo no Kimyou na Bouken Part 6: Stone Ocean,8.55
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gintama.: Porori-hen,8.53
292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,One Piece Movie 14: Stampede,8.23


In [13]:
nulos2 = df.loc[df['Tipo'].isna() == True] 
display(nulos2)

df.drop([181,183,223,226,387],axis=0,inplace=True) # Foram retirados por serem músicas

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
181,Music,NaN,add some,NaN,Other,1.0,NaN,NaN,Finished Airing,PG-13 - Teens 13 or older,NaN,Yoru Ni Kakeru,8.36
183,Music,Fantasy,Bones,NaN,Game,1.0,NaN,NaN,Finished Airing,PG - Children,NaN,Gotcha!,8.36
223,Music,NaN,add some,NaN,Music,1.0,NaN,NaN,Finished Airing,PG-13 - Teens 13 or older,NaN,Kawaki wo Ameku,8.31
226,Music,Sci-Fi,A-1 Pictures,NaN,Music,1.0,NaN,NaN,Finished Airing,G - All Ages,NaN,Shelter (Music),8.31
387,Music,"Adventure, Drama, Sci-Fi",Toei Animation,NaN,Music,1.0,NaN,NaN,Finished Airing,G - All Ages,NaN,Interstella5555: The 5tory of The 5ecret 5tar ...,8.14


In [14]:
nulos3 = df.loc[df['Episódios'].isna() == True]
display(nulos3)

df.drop(151,axis=0,inplace=True)

df.loc[68,'Episódios'] = 1013 #Colocando os valores manualmente 
df.loc[366, 'Episódios'] = 123

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
68,Super Power,"Action, Adventure, Comedy, Drama, Fantasy",Toei Animation,Shounen,Manga,NaN,Fall 1999,Sundays at 09:30 (JST),Currently Airing,PG-13 - Teens 13 or older,TV,One Piece,8.62
151,NaN,Comedy,Hololive Production,NaN,Other,NaN,NaN,NaN,Currently Airing,PG-13 - Teens 13 or older,ONA,Holo no Graffiti,8.41
366,Police,"Adventure, Comedy, Mystery",TMS Entertainment,Shounen,Manga,NaN,Winter 1996,Saturdays at 18:00 (JST),Currently Airing,PG-13 - Teens 13 or older,TV,Detective Conan,8.16


In [15]:
nulos4 = df.loc[df['Gêneros'].isna() == True]
display(nulos4)

df.drop([176,191,349,464,475],axis=0,inplace=True)

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
176,Vampire,NaN,Shaft,NaN,Light novel,1.0,NaN,NaN,Finished Airing,R - 17+ (violence & profanity),Movie,Kizumonogatari I: Tekketsu-hen,8.38
191,"Music, School",NaN,Kinema Citrus,NaN,Original,1.0,NaN,NaN,Finished Airing,PG-13 - Teens 13 or older,Movie,Shoujo☆Kageki Revue Starlight Movie,8.35
349,"Military, School",NaN,Actas,NaN,Original,1.0,NaN,NaN,Finished Airing,PG-13 - Teens 13 or older,Movie,Girls & Panzer: Saishuushou Part 3,8.18
464,Music,NaN,Hololive Production,NaN,Other,1.0,NaN,NaN,Finished Airing,G - All Ages,ONA,Hololive Alternative,8.08
475,"Military, School",NaN,Actas,NaN,Original,1.0,NaN,NaN,Finished Airing,PG-13 - Teens 13 or older,Movie,Girls & Panzer: Saishuushou Part 2,8.07


In [26]:
nulos5 = df.isna().sum() # Quantidade de nulos após o tratamento
display(nulos5)

df.to_excel('df_limpo.xlsx') #Salvando o novo DF

Tema               146
Gêneros              0
Estúdio              0
Demografia         214
Origem               0
Episódios            0
Estréia            192
Horário exibido    192
Status               0
Classificação        0
Tipo                 0
Título               0
Pontuação            0
dtype: int64

#### Carregando o dataframe atualizado:

In [139]:
df_limpo = pd.read_excel('df_limpo.xlsx',index_col='Rank')
display(df_limpo.head())

,Tema,Gêneros,Estúdio,Demografia,Origem,Episódios,Estréia,Horário exibido,Status,Classificação,Tipo,Título,Pontuação
Rank,,,,,,,,,,,,,
1,Military,"Action, Adventure, Comedy, Drama, Fantasy",Bones,Shounen,Manga,64,Spring 2009,Sundays at 17:00 (JST),Finished Airing,R - 17+ (violence & profanity),TV,Fullmetal Alchemist: Brotherhood,9.15
2,"Military, Super Power","Action, Drama, Fantasy, Mystery",MAPPA,Shounen,Manga,12,Winter 2022,Mondays at 00:05 (JST),Currently Airing,R - 17+ (violence & profanity),TV,Shingeki no Kyojin: The Final Season Part 2,9.13
3,Psychological,"Drama, Sci-Fi, Suspense",White Fox,NaN,Visual novel,24,Spring 2011,Wednesdays at 02:05 (JST),Finished Airing,PG-13 - Teens 13 or older,TV,Steins;Gate,9.09
4,"Historical, Parody, Samurai","Action, Comedy, Sci-Fi",Bandai Namco Pictures,Shounen,Manga,51,Spring 2015,Wednesdays at 18:00 (JST),Finished Airing,PG-13 - Teens 13 or older,TV,Gintama°,9.09
5,"Military, Super Power","Action, Drama, Fantasy, Mystery",Wit Studio,Shounen,Manga,10,Spring 2019,Mondays at 00:10 (JST),Finished Airing,R - 17+ (violence & profanity),TV,Shingeki no Kyojin Season 3 Part 2,9.08
